# Biostat 257 Homework 6

**Due June 10 @ 11:59PM**

In [1]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin19.5.0)
  CPU: Intel(R) Core(TM) i5-8279U CPU @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)


Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma}_i + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma}_i \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW5, we used the nonlinear programming (NLP) approach (Newton type algorithms) for optimization. In this assignment, we derive and implement an expectation-maximization (EM) algorithm for the same problem.

In [2]:
# load necessary packages; make sure install them first
using BenchmarkTools, Distributions, LinearAlgebra, Random, Revise
import LinearAlgebra: BlasReal, copytri!

## Q1. (10 pts) Refresher on normal-normal model

Assume the conditional distribution
$$
\mathbf{y} \mid \boldsymbol{\gamma} \sim N(\mathbf{X} \boldsymbol{\beta} + \mathbf{Z} \boldsymbol{\gamma}, \sigma^2 \mathbf{I}_n)
$$
and the prior distribution
$$
\boldsymbol{\gamma} \sim N(\mathbf{0}_q, \boldsymbol{\Sigma}).
$$
By the Bayes theorem, the posterior distribution is
\begin{eqnarray*}
f(\boldsymbol{\gamma} \mid \mathbf{y}) &=& \frac{f(\mathbf{y} \mid \boldsymbol{\gamma}) \times f(\boldsymbol{\gamma})}{f(\mathbf{y})}, \end{eqnarray*}
where $f$ denotes corresponding density. 

Show that the posterior distribution of random effects $\boldsymbol{\gamma}$ is a multivariate normal with mean
\begin{eqnarray*}
\mathbb{E} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& \sigma^{-2} (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1 } \mathbf{Z}^T (\mathbf{y} - \mathbf{X} \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} (\mathbf{y} - \mathbf{X} \boldsymbol{\beta})
\end{eqnarray*}
and covariance
\begin{eqnarray*}
\text{Var} (\boldsymbol{\gamma} \mid \mathbf{y}) &=& (\sigma^{-2} \mathbf{Z}^T \mathbf{Z} + \boldsymbol{\Sigma}^{-1})^{-1} \\
&=& \boldsymbol{\Sigma} - \boldsymbol{\Sigma} \mathbf{Z}^T (\mathbf{Z} \boldsymbol{\Sigma} \mathbf{Z}^T + \sigma^2 \mathbf{I})^{-1} \mathbf{Z} \boldsymbol{\Sigma}.
\end{eqnarray*}

### Answer

\begin{eqnarray*}
f(\gamma\vert y) &=& \frac{f(y\vert \gamma)f(\gamma)}{f(y)}\\
 &\propto& f(y\vert \gamma)f(y)\\
 &\propto& \sigma^{-n}\lvert\Sigma\rvert^{-1/2}\exp(-\frac{1}{2}[\frac{1}{\sigma^{2}}(y - X\beta - Z\gamma)^{T}(y - X\beta - Z\gamma) + \gamma^{T}\Sigma^{-1}\gamma])
\end{eqnarray*}

For convenience, rewrite $r = y - X\beta$. Then,
\begin{eqnarray*}
f(\gamma\vert y) &\propto& \sigma^{-n}\lvert\Sigma\rvert^{-1/2}\exp(-\frac{1}{2}[\frac{1}{\sigma^{2}}(y - X\beta - Z\gamma)^{T}(y - X\beta - Z\gamma) + \gamma^{T}\Sigma^{-1}\gamma])\\
 &=& \sigma^{-n}\lvert\Sigma\rvert^{-1/2}\exp(-\frac{1}{2}[\frac{1}{\sigma^{2}}(r - Z\gamma)^{T}(r - Z\gamma) + \gamma^{T}\Sigma^{-1}\gamma])\\
 &=& \sigma^{-n}\lvert\Sigma\rvert^{-1/2}\exp(-\frac{1}{2}[\frac{1}{\sigma^{2}}(r^{T}r - 2r^{T}Z\gamma + \gamma^{T}Z^{T}Z\gamma) + \gamma^{T}\Sigma^{-1}\gamma])\\
 &\propto& \sigma^{-n}\lvert\Sigma\rvert^{-1/2}\exp(-\frac{1}{2}[(\gamma^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})\gamma - 2\gamma^{T}(\sigma^{-2}Z^{T}Z + \Sigma^{-1})(\sigma^{-2}Z^{T}Z + \Sigma^{-1})^{-1}\frac{Z^{T}r}{\sigma^{2}}])\\
\end{eqnarray*}

This implies that $\gamma\vert y \sim N((\sigma^{-2}Z^{T}Z + \Sigma^{-1})^{-1}\frac{Z^{T}r}{\sigma^{2}}, (\sigma^{-2}Z^{T}Z + \Sigma^{-1})^{-1})$. The mean and variance being apparent from the form of this distribution demonstrates what we've set out to prove. $
\square$

## Q2. (20 pts) Derive EM algorithm

1. Write down the complete log-likelihood
$$
\sum_{i=1}^m \log f(\mathbf{y}_i, \boldsymbol{\gamma}_i \mid \boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)
$$

2. Derive the $Q$ function (E-step).
$$
Q(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2 \mid \boldsymbol{\beta}^{(t)}, \boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)}).
$$

3. Derive the EM (or ECM) update of $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$ (M-step). 

### Answer

1. We begin with the expression for the log-likelihood of a single point:
\begin{eqnarray*}
f(y_{i},\gamma_{i}\vert \beta, \Sigma, \sigma^{2}) &=& f(y_{i}\vert \gamma_{i},\beta,\Sigma,\sigma^{2})f(\gamma_{i})\\
 &=& (2\pi)^{-\frac{n_{i} + q}{2}}\sigma^{-n_{i}}\lvert\Sigma\rvert^{-1/2}\exp(-\frac{1}{2}[\frac{1}{\sigma^{2}}(y_{i} - X_{i}\beta - Z_{i}\gamma_{i})^{T}(y_{i} - X_{i}\beta - Z_{i}\gamma_{i}) + \gamma_{i}^{T}\Sigma^{-1}\gamma_{i}])\\
\log f(y_{i},\gamma_{i}\vert \beta, \Sigma, \sigma^{2}) &=& -\frac{n_{i} + q}{2}\log(2\pi) - \frac{n_{i}}{2}\log\sigma^{2} - \frac{1}{2}\log\lvert\Sigma\rvert -\frac{1}{2}[\frac{1}{\sigma^{2}}(y_{i} - X_{i}\beta - Z_{i}\gamma_{i})^{T}(y_{i} - X_{i}\beta - Z_{i}\gamma_{i}) + \gamma_{i}^{T}\Sigma^{-1}\gamma_{i}]
\end{eqnarray*}

We can simplify the expression more to make it amenable to coding:

\begin{eqnarray*}
\log f(y_{i},\gamma_{i}\vert \beta, \Sigma, \sigma^{2}) &=& -\frac{n_{i} + q}{2}\log(2\pi) - \frac{n_{i}}{2}\log\sigma^{2} - \frac{1}{2}\log\lvert\Sigma\rvert\\
 && -\frac{1}{2}[\sigma^{-2}(y_{i} - X_{i}\beta)^{T}(y_{i} - X_{i}\beta) - 2\sigma^{-2}(y_{i} - X_{i}\beta)^{T}Z_{i}\gamma_{i} + \gamma_{i}^{T}(Z_{i}^{T}Z_{i}\sigma^{-2} + \Sigma^{-1})\gamma_{i}]\\
 &=& -\frac{n_{i} + q}{2}\log(2\pi) - \frac{n_{i}}{2}\log\sigma^{2} - \frac{1}{2}\log\lvert\Sigma\rvert\\
 && -\frac{1}{2}[\sigma^{-2}(y_{i}^{T}y_{i} - 2\beta^{T}X_{i}^{T}y_{i} + \beta^{T}X_{i}^{T}X_{i}\beta) - 2\sigma^{-2}(y_{i} - X_{i}\beta)^{T}Z_{i}\gamma_{i}\\
 && + \sigma^{-2}\gamma_{i}^{T}L^{T,-1}(L^{T}Z_{i}^{T}Z_{i}L + \sigma^{2}I_{q})L^{-1}\gamma_{i}]\\
 &=& -\frac{n_{i} + q}{2}\log(2\pi) - \frac{n_{i}}{2}\log\sigma^{2} - \frac{1}{2}\log\lvert\Sigma\rvert\\
 && -\frac{1}{2}[\sigma^{-2}(y_{i}^{T}y_{i} - 2\beta^{T}X_{i}^{T}y_{i} + \beta^{T}X_{i}^{T}X_{i}\beta) - 2\sigma^{-2}(y_{i} - X_{i}\beta)^{T}Z_{i}\gamma_{i}\\
 && + \sigma^{-2}\gamma_{i}^{T}L^{T,-1}M_{i}L^{-1}\gamma_{i}]\\
\end{eqnarray*}

where $M_{i} = L^{T}Z_{i}^{T}Z_{i}L + \sigma^{2}I_{q}$.

Summing over $i = 1$ to $m$ gets us the complete log-likelihood:
\begin{eqnarray*}
\sum_{i=1}^{m}\log f(y_{i},\gamma_{i}\vert \beta, \Sigma, \sigma^{2}) &=& - \frac{m}{2}\log\lvert\Sigma\rvert - \frac{1}{2}\sum_{i=1}^{m}[(n_{i} + q)\log(2\pi) + n_{i}\log\sigma^{2} + \frac{1}{\sigma^{2}}(y_{i} - X_{i}\beta - Z_{i}\gamma_{i})^{T}(y_{i} - X_{i}\beta - Z_{i}\gamma_{i}) + \gamma_{i}^{T}\Sigma^{-1}\gamma_{i}]\\
 &=& -\frac{m}{2}\log\lvert\Sigma\rvert - \frac{1}{2}\sum_{i=1}^{m}[(n_{i} + q)\log(2\pi) + n_{i}\log\sigma^{2} + \frac{1}{\sigma^{2}}((y_{i} - X_{i}\beta)^{T}(y_{i} - X_{i}\beta) - 2(y_{i} - X_{i}\beta)^{T}Z_{i}\gamma_{i}) + \gamma_{i}^{T}(Z^{T}Z\sigma^{-2} + \Sigma^{-1})\gamma_{i}]\\
\end{eqnarray*}

2. Treat $\gamma_{i}$ as our missing data. Then our $Q$ function takes the following form:
\begin{eqnarray*}
Q(\beta,\Sigma,\sigma^{2}\vert \beta^{(t)}, \Sigma^{(t)}, \sigma^{2(t)}) &=& \mathbb{E}[\sum_{i=1}^{m}\log f(y_{i},\gamma_{i}\vert \beta, \Sigma, \sigma^{2})\vert y, \beta^{(T)}, \Sigma^{(t)}, \sigma^{2(t)}]\\
 &=& - \frac{m}{2}\log\lvert\Sigma\rvert - \frac{1}{2}\sum_{i=1}^{m}[(n_{i} + q)\log(2\pi) + n_{i}\log\sigma^{2} + \frac{1}{\sigma^{2}}((y_{i} - X_{i}\beta)^{T}(y_{i} - X_{i}\beta)\\
 &&- 2(y_{i} - X_{i}\beta)^{T}Z_{i}\mathbb{E}(\gamma_{i}\vert y_{i}) + \mathrm{tr}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})(Z^{T}Z\sigma^{-2} + \Sigma^{-1}))]\\
\end{eqnarray*}



3. We take the derivative of $Q$ with respect to each of the three parameters we aim to estimate:
\begin{eqnarray*}
\frac{\partial Q}{\partial \beta} &=& - \frac{1}{2}\sum_{i=1}^{m}[\frac{1}{\sigma^{2}}(2X_{i}^{T}X_{i}\beta - 2X_{i}^{T}y + 2X_{i}^{T}Z_{i}(\sigma^{-2(t)}Z_{i}^{T}Z_{i} + \Sigma^{-1(t)})^{-1}\frac{Z^{T}_{i}(y_{i} - X_{i}\beta^{(t)})}{\sigma^{2(t)}})] = 0\\
\sum_{i=1}^{m}X_{i}^{T}X_{i}\beta &=& \sum_{i=1}^{m}[X_{i}^{T}y - X_{i}^{T}Z_{i}(\sigma^{-2(t)}Z_{i}^{T}Z_{i} + \Sigma^{-1(t)})^{-1}\frac{Z^{T}_{i}(y_{i} - X_{i}\beta^{(t)})}{\sigma^{2(t)}})]\\
\beta^{(t+1)} &=& \left[\sum_{i=1}^{m}X_{i}^{T}X_{i}\right]^{-1}\left[\sum_{i=1}^{m}[X_{i}^{T}y - X_{i}^{T}Z_{i}\mathbb{E}(\gamma_{i}\vert y_{i})]\right]\\
\end{eqnarray*}

We use the matrix differential operator $D$ described in Dr. Hua Zhou's BIOSTAT 216 course in 2019 to acquire $\Sigma^{t+1}$:
\begin{eqnarray*}
D_{\Sigma}(Q) &=& -\frac{m}{2}D_{\Sigma}\log\lvert\Sigma\rvert - \frac{1}{2}\sum_{i=1}^{m}[(D_{\Sigma}\mathrm{tr}((\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})(Z^{T}Z\sigma^{-2} + \Sigma^{-1}))]\\
 &=& -\frac{m}{2\lvert\Sigma\rvert}(\mathrm{vec}(\lvert\Sigma\rvert\Sigma^{-1}))^{T} - \frac{1}{2}\sum_{i=1}^{m}\left[\mathrm{vec}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})^{T}D_{\Sigma}(Z^{T}Z\sigma^{-2} + \Sigma^{-1})\right]\\
 &=& -\frac{m}{2}(\mathrm{vec}(\Sigma^{-1}))^{T} - \frac{1}{2}\sum_{i=1}^{m}\left[\mathrm{vec}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})^{T}(-\Sigma^{-1} \otimes \Sigma^{-1})\right]\\
 &=& -\frac{m}{2}(\mathrm{vec}(\Sigma^{-1}))^{T} + \frac{1}{2}\sum_{i=1}^{m}\left[\mathrm{vec}(\Sigma^{-1}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})\Sigma^{-1})^{T}\right] = 0\\
\end{eqnarray*}

Since this equality holds for the vectorization of the expression, we can set the insides of the vectorization to zero as well:

\begin{eqnarray*}
-\frac{m}{2}\Sigma^{-1} + \frac{1}{2}\sum_{i=1}^{m}\Sigma^{-1}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})\Sigma^{-1} &=& 0\\
-mI_{q} + \sum_{i=1}^{m}\Sigma^{-1}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T}) &=& 0\\
\Sigma^{(t+1)} &=& \frac{1}{m}\sum_{i=1}^{m}(\mathrm{Var}(\gamma_{i}\vert y_{i}) + \mathbb{E}(\gamma_{i}\vert y_{i})\mathbb{E}(\gamma_{i}\vert y_{i})^{T})
\end{eqnarray*}

Finally, we differentiate by $\sigma^{2}$:
\begin{eqnarray*}
\frac{\partial Q}{\partial \sigma^{2}} &=& - \frac{1}{2}\sum_{i=1}^{m}[\frac{n_{i}}{\sigma^{2}} -\frac{1}{\sigma^{4}}((y_{i} - X_{i}\beta)^{T}(y_{i} - X_{i}\beta) - 2(y_{i} - X_{i}\beta)^{T}Z_{i}(\sigma^{-2(t)}Z_{i}^{T}Z_{i} + \Sigma^{-1(t)})^{-1}\frac{Z^{T}_{i}(y_{i} - X_{i}\beta^{(t)})}{\sigma^{2(t)}})\\
&&- \sigma^{-4}\mathrm{tr}((\sigma^{-2(t)}Z_{i}^{T}Z_{i} + \Sigma^{-1(t)})^{-1}(Z_{i}^{T}Z_{i}))] = 0\\
\sigma^{2(t+1)} &=&  \frac{1}{\sum_{i=1}^{m}n_{i}}\sum_{i=1}^{m}[(y_{i} - X_{i}\beta)(y_{i} - X_{i}\beta) - \frac{2}{\sigma^{2(t)}}(y_{i} - X_{i}\beta)^{T}Z_{i}(\sigma^{-2(t)}Z_{i}^{T}Z_{i} + \Sigma^{-1(t)})^{-1}Z^{T}_{i}(y_{i} - X_{i}\beta^{(t)}) + \mathrm{tr}((\sigma^{-2(t)}Z_{i}^{T}Z_{i} + \Sigma^{-1(t)})^{-1}(Z_{i}^{T}Z_{i}))]
\end{eqnarray*}

(A careless mistake caught and more efficient notation revised with help from Kelly Li.)

## Q3. (20 pts) Objective of a single datum

We modify the code from HW5 to evaluate the objective, the conditional mean of $\boldsymbol{\gamma}$, and the conditional variance of $\boldsymbol{\gamma}$. Start-up code is provided below. You do _not_ have to use this code.

In [3]:
# define a type that holds an LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y          :: Vector{T}
    X          :: Matrix{T}
    Z          :: Matrix{T}
    # posterior mean and variance of random effects γ
    μγ         :: Vector{T} # posterior mean of random effects
    νγ         :: Matrix{T} # posterior variance of random effects
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    yty        :: T
    rtr        :: Vector{T}
    xty        :: Vector{T}
    zty        :: Vector{T}
    ztr        :: Vector{T}
    ltztr      :: Vector{T}
    xtr        :: Vector{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    ltztzl     :: Matrix{T}
    storage_qq :: Matrix{T}
end

"""
    LmmObs(y::Vector, X::Matrix, Z::Matrix)

Create an LMM datum of type `LmmObs`.
"""
function LmmObs(
    y::Vector{T}, 
    X::Matrix{T}, 
    Z::Matrix{T}) where T <: AbstractFloat
    n, p, q = size(X, 1), size(X, 2), size(Z, 2)
    μγ         = Vector{T}(undef, q)
    νγ         = Matrix{T}(undef, q, q)
    yty        = abs2(norm(y))
    rtr        = Vector{T}(undef, 1)
    xty        = transpose(X) * y
    zty        = transpose(Z) * y
    ztr        = similar(zty)
    ltztr      = similar(zty)
    xtr        = Vector{T}(undef, p)
    storage_p  = similar(xtr)
    storage_q  = Vector{T}(undef, q)
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    ltztzl     = similar(ztz)
    storage_qq = similar(ztz)
    LmmObs(y, X, Z, μγ, νγ, 
        yty, rtr, xty, zty, ztr, ltztr, xtr,
        storage_p, storage_q, 
        xtx, ztx, ztz, ltztzl, storage_qq)
end

"""
    logl!(obs::LmmObs, β, Σ, L, σ², updater = false)

Evaluate the log-likelihood of a single LMM datum at parameter values `β`, `Σ`, 
and `σ²`. The lower triangular Cholesky factor `L` of `Σ` must be supplied too.
The fields `obs.μγ` and `obs.νγ` are overwritten by the posterior mean and 
posterior variance of random effects. If `updater==true`, fields `obs.ztr`, 
`obs.xtr`, and `obs.rtr` are updated according to input parameter values. 
Otherwise, it assumes these three fields are pre-computed. 
"""
function logl!(
        obs     :: LmmObs{T}, 
        β       :: Vector{T}, 
        Σ       :: Matrix{T},
        L       :: Matrix{T},
        σ²      :: T,
        updater :: Bool = false
        ) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)
    σ²inv   = inv(σ²)
    ####################
    # Evaluate objective
    ####################
    # form the q-by-q matrix: Lt Zt Z L
    copy!(obs.ltztzl, obs.ztz)
    BLAS.trmm!('L', 'L', 'T', 'N', T(1), L, obs.ltztzl) # O(q^3)
    BLAS.trmm!('R', 'L', 'N', 'N', T(1), L, obs.ltztzl) # O(q^3)        
    # form the q-by-q matrix: M = σ² I + Lt Zt Z L
    copy!(obs.storage_qq, obs.ltztzl)
    @inbounds for j in 1:q
        obs.storage_qq[j, j] += σ²
    end
    LAPACK.potrf!('U', obs.storage_qq) # O(q^3)
    # Zt * res
    updater && BLAS.gemv!('N', T(-1), obs.ztx, β, T(1), copy!(obs.ztr, obs.zty)) # O(pq)
    # Lt * (Zt * res)
    BLAS.trmv!('L', 'T', 'N', L, copy!(obs.ltztr, obs.ztr))    # O(q^2)
    # storage_q = (Mchol.U') \ (Lt * (Zt * res))
    BLAS.trsv!('U', 'T', 'N', obs.storage_qq, copy!(obs.storage_q, obs.ltztr)) # O(q^3)
    # Xt * res = Xt * y - Xt * X * β
    updater && BLAS.gemv!('N', T(-1), obs.xtx, β, T(1), copy!(obs.xtr, obs.xty))
    # l2 norm of residual vector
    updater && (obs.rtr[1] = obs.yty - dot(obs.xty, β) - dot(obs.xtr, β))
    # assemble pieces
    logl::T = n * log(2π) + (n - q) * log(σ²) # constant term
    @inbounds for j in 1:q # log det term
        logl += 2log(obs.storage_qq[j, j])
    end
    qf    = abs2(norm(obs.storage_q)) # quadratic form term
    logl += (obs.rtr[1] - qf) * σ²inv 
    logl /= -2
    ######################################
    # Evaluate posterior mean and variance
    ######################################   
    # posterior mean
    copy!(obs.μγ, obs.ltztr)
    LAPACK.potrs!('U', obs.storage_qq, obs.μγ)
    BLAS.trmv!('L', 'N', 'N', L, obs.μγ)
    
    # posterior variance
    copy!(obs.νγ, transpose(L))
    LAPACK.potrs!('U', obs.storage_qq, obs.νγ)
    BLAS.trmm!('L', 'L', 'N', 'N', one(T), L, obs.νγ)
    obs.νγ .*= σ²
    
    ###################
    # Return
    ###################        
    return logl
end


logl!

It is a good idea to test correctness and efficiency of the single datum objective/posterior mean/var evaluator here. It's the same test datum in HW3 and HW5.

In [4]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X = [ones(n) randn(n, p - 1)]
Z = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I # compound symmetry 
L  = Matrix(cholesky(Symmetric(Σ)).L)
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form the LmmObs object
obs = LmmObs(y, X, Z);

### Correctness

In [5]:
@show logl = logl!(obs, β, Σ, L, σ², true)
@show obs.μγ
@show obs.νγ;

logl = logl!(obs, β, Σ, L, σ², true) = -3256.1793358058258
obs.μγ = [0.10608689301332817, -0.2510419060257254, -1.4653979409850375]
obs.νγ = [0.0007494356395786613 -1.2183420464422381e-6 -2.176783682941727e-6; -1.2183420464422335e-6 0.0007542331466357766 2.1553464612468783e-5; -2.1767836829417363e-6 2.1553464612468773e-5 0.000746527134491723]


You will lose all 20 points if following statement throws `AssertionError`.

In [6]:
@assert abs(logl - (-3256.1793358058258)) < 1e-4
@assert norm(obs.μγ - [0.10608689301333621, 
        -0.25104190602577225, -1.4653979409855415]) < 1e-4
@assert norm(obs.νγ - [
        0.0007494356395909563 -1.2183420093769967e-6 -2.176783643112221e-6; 
        -1.2183420282298223e-6 0.0007542331467601107 2.1553464632686345e-5; 
        -2.1767836636008638e-6 2.1553464641863096e-5 0.0007465271342535443
        ]) < 1e-4

### Efficiency

Benchmark for efficiency.

In [7]:
bm_obj = @benchmark logl!($obs, $β, $Σ, $L, $σ², true)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.666 μs …  11.433 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.708 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.791 μs ± 462.588 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▅ ▁▂▂ ▁                                                   ▂
  █████████▅▅▄▁▄▃▄▁▃▁▃▁▃▁▁▁▃▁▁▃▁▁▁▁▁▃▄▁▃▃▅▇▇▆▇▄▅▆▇▆▇▆▄▃▅▄▁▄▃▄ █
  1.67 μs      Histogram: log(frequency) by time      4.07 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median runt time is 1.8μs. You will get full credit if the median run time is within 10μs. The points you will get are

In [8]:
clamp(10 / (median(bm_obj).time / 1e3) * 10, 0, 10)

10.0

In [9]:
# # check for type stability
# @code_warntype logl!(obs, β, Σ, L, σ²)

In [10]:
# using Profile

# Profile.clear()
# @profile for i in 1:10000; logl!(obs, β, Σ, L, σ²); end
# Profile.print(format=:flat)

## Q4. LmmModel type

We modify the `LmmModel` type in HW4 to hold all data points, model parameters, and intermediate arrays.

In [11]:
# define a type that holds LMM model (data + parameters)
struct LmmModel{T <: AbstractFloat}
    # data
    data :: Vector{LmmObs{T}}
    # parameters
    β      :: Vector{T}
    Σ      :: Matrix{T}
    L      :: Matrix{T}
    σ²     :: Vector{T}    
    # TODO: add whatever intermediate arrays you may want to pre-allocate
    xty    :: Vector{T}
    xtr    :: Vector{T}
    ztr2   :: Vector{T}
    xtxinv :: Matrix{T}
    ztz2   :: Matrix{T}
    storage_p :: Vector{T}
    storage_qq :: Matrix{T}
end

"""
    LmmModel(data::Vector{LmmObs})

Create an LMM model that contains data and parameters.
"""
function LmmModel(obsvec::Vector{LmmObs{T}}) where T <: AbstractFloat
    # dims
    p      = size(obsvec[1].X, 2)
    q      = size(obsvec[1].Z, 2)
    # parameters
    β      = Vector{T}(undef, p)
    Σ      = Matrix{T}(undef, q, q)
    L      = Matrix{T}(undef, q, q)
    σ²     = Vector{T}(undef, 1)    
    # intermediate arrays
    xty    = zeros(T, p)
    xtr    = similar(xty)
    ztr2   = Vector{T}(undef, abs2(q))
    xtxinv = zeros(T, p, p)
    # pre-calculate \sum_i Xi^T Xi and \sum_i Xi^T y_i
    @inbounds for i in eachindex(obsvec)
        obs = obsvec[i]
        BLAS.axpy!(T(1), obs.xtx, xtxinv)
        BLAS.axpy!(T(1), obs.xty, xty)
    end
    # invert X'X
    LAPACK.potrf!('U', xtxinv)
    LAPACK.potri!('U', xtxinv)
    LinearAlgebra.copytri!(xtxinv, 'U')
    ztz2   = Matrix{T}(undef, abs2(q), abs2(q))
    storage_p = Vector{T}(undef, p)
    storage_qq = Matrix{T}(undef, q, q)
    LmmModel(obsvec, β, Σ, L, σ², xty, xtr, ztr2, xtxinv, ztz2, 
        storage_p, storage_qq)
end

LmmModel

## Q5. Implement EM update

Let's write the key function `update_em!` that performs one iteration of EM update.

In [12]:
"""
    update_em!(m::LmmModel, updater::Bool = false)

Perform one iteration of EM update. It returns the log-likelihood calculated 
from input `m.β`, `m.Σ`, `m.L`, and `m.σ²`. These fields are then overwritten 
by the next EM iterate. The fields `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` are updated according to the resultant `m.β`. If `updater==true`, 
the function first updates `m.data[i].xtr`, `m.data[i].ztr`, and 
`m.data[i].rtr` according to `m.β`. If `updater==false`, it assumes these fields 
are pre-computed.
"""
function update_em!(m::LmmModel{T}, updater::Bool = false) where T <: AbstractFloat
    logl = zero(T)
    fill!(m.storage_p, 0)
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        # initialize stored values. Is this necessary???
        logl += logl!(obs, m.β, m.Σ, m.L, m.σ²[1], updater)
    
        # update m.β relevant figures
        BLAS.gemv!('T', -one(T), obs.ztx, obs.μγ, one(T), copy!(obs.storage_p, obs.xty))
        m.storage_p .+= obs.storage_p
    end
    
    # update m.β
    mul!(m.β, m.xtxinv, m.storage_p)
    
    if updater
        # update m.data[i].ztr, m.data[i].xtr, m.data[i].rtr
        @inbounds for i in eachindex(m.data)
            obs = m.data[i]
            BLAS.gemv!('N', -one(T), obs.ztx, m.β, one(T), copy!(obs.ztr, obs.zty))
            BLAS.gemv!('N', -one(T), obs.xtx, m.β, one(T), copy!(obs.xtr, obs.xty))
            obs.rtr[1] = obs.yty - 2 * dot(m.β, obs.xty) + dot(m.β, obs.xtx, m.β)
#            BLAS.trmv!('L', 'T', 'N', m.L, copy!(obs.ltztr, obs.ztr))
            # update other terms as well???
            #logl!(obs, m.β, m.Σ, m.L, m.σ²[1], updater)
        end
    end
    
    fill!(m.storage_qq, 0)
    s2 = 0
    nsum = 0
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        # update m.σ² constants
        # acquire M⁻¹
        #TODO: debug?
        LAPACK.potri!('U', obs.storage_qq)
        # aggregate figures
        rtr_new = obs.yty - 2 * dot(m.β, obs.xty) + dot(m.β, obs.xtx, m.β)
        # Zi'ri. Update with new β to mirror our expression.
        BLAS.gemv!('N', -one(T), obs.ztx, m.β, one(T), copy!(obs.storage_q, obs.zty))
#        BLAS.trmv!('L', 'T', 'N', m.L, obs.storage_q)
#        BLAS.trmm!('L', 'L', 'N', 'N', one(T), m.L, obs.storage_qq)
#        BLAS.trmm!('R', 'L', 'T', 'N', one(T), m.L, obs.storage_qq)
        
        s2 += rtr_new - 2 * dot(obs.storage_q, obs.μγ) +
            dot(obs.νγ, obs.ztz) + dot(obs.μγ, obs.ztz, obs.μγ)
        nsum += length(obs.y)
    
        # update m.Σ and m.L
        m.storage_qq .+= obs.νγ
        BLAS.syrk!('U', 'N', one(T), obs.μγ, one(T), m.storage_qq)
    end
    
    m.σ² .= s2 / nsum
    
    copytri!(m.storage_qq, 'U')
    copy!(m.Σ, m.storage_qq)
    m.Σ ./= length(m.data)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    q = size(m.Σ, 1)
    for i in 1:(q-1)
        for j in (i + 1):q
            m.L[i, j] = 0
        end
    end
        
    # return log-likelihood at input parameter values
    logl
end

update_em!

## Q6. (30 pts) Test data

Let's generate a fake longitudinal data set (same as HW4) to test our algorithm.

In [13]:
Random.seed!(257)

# dimension
m      = 1000 # number of individuals
ns     = rand(1500:2000, m) # numbers of observations per individual
p      = 5 # number of fixed effects, including intercept
q      = 3 # number of random effects, including intercept
obsvec = Vector{LmmObs{Float64}}(undef, m)
# true parameter values
βtrue  = [0.1; 6.5; -3.5; 1.0; 5]
σ²true = 1.5
σtrue  = sqrt(σ²true)
Σtrue  = Matrix(Diagonal([2.0; 1.2; 1.0]))
Ltrue  = Matrix(cholesky(Symmetric(Σtrue)).L)
# generate data
for i in 1:m
    # first column intercept, remaining entries iid std normal
    X = Matrix{Float64}(undef, ns[i], p)
    X[:, 1] .= 1
    @views Distributions.rand!(Normal(), X[:, 2:p])
    # first column intercept, remaining entries iid std normal
    Z = Matrix{Float64}(undef, ns[i], q)
    Z[:, 1] .= 1
    @views Distributions.rand!(Normal(), Z[:, 2:q])
    # generate y
    y = X * βtrue .+ Z * (Ltrue * randn(q)) .+ σtrue * randn(ns[i])
    # form a LmmObs instance
    obsvec[i] = LmmObs(y, X, Z)
end
# form a LmmModel instance
lmm = LmmModel(obsvec);

### Correctness

Evaluate log-likelihood and gradient at the true parameter values.

In [14]:
copy!(lmm.β, βtrue)
copy!(lmm.Σ, Σtrue)
copy!(lmm.L, Ltrue)
lmm.σ²[1] = σ²true
@show obj1 = update_em!(lmm, true)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²
println()
@show obj2 = update_em!(lmm, false)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

obj1 = update_em!(lmm, true) = -2.840068438369969e6
lmm.β = [0.10003613673625025, 6.500382871080186, -3.4998646342112356, 0.9997124657606635, 4.999230851463537]
lmm.Σ = [1.9903882760455245 0.06862095707037451 0.05347290179472648; 0.06862095707037451 1.2813220461216892 -0.09044913324906331; 0.05347290179472648 -0.09044913324906331 0.9435400745724167]
lmm.L = [1.410811212049835 0.0 0.0; 0.04863936186803607 1.1309094829378519 0.0; 0.03790223761904553 -0.08160924927472397 0.9671832429217858]
lmm.σ² = [1.4987367279243653]

obj2 = update_em!(lmm, false) = -2.84006046054206e6
lmm.β = [0.10007136571174438, 6.500383550631524, -3.4998642980415418, 0.9997119269497654, 4.999229480978786]
lmm.Σ = [1.990377538078367 0.06870107683947124 0.053543517504439274; 0.06870107683947124 1.2814409213711735 -0.09059223997463767; 0.053543517504439274 -0.09059223997463767 0.9434431688085722]
lmm.L = [1.4108074064443974 0.0 0.0; 0.0486962830827603 1.1309595896339961 0.0; 0.037952393260666885 -0.08173623647370382 0

1-element Vector{Float64}:
 1.4987345544160287

Test correctness. You will loss all 30 points if following code throws `AssertError`.

In [15]:
@assert abs(obj1 - (-2.840068438369969e6)) < 1e-4
@assert abs(obj2 - (-2.84006046054206e6)) < 1e-4

### Efficiency

Test efficiency of EM update.

In [16]:
bm_emupdate = @benchmark update_em!($lmm, true) setup=(
    copy!(lmm.β, βtrue);
    copy!(lmm.Σ, Σtrue);
    copy!(lmm.L, Ltrue);
    lmm.σ²[1] = σ²true)

BenchmarkTools.Trial: 1567 samples with 1 evaluation.
 Range (min … max):  2.448 ms …  15.539 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.006 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.182 ms ± 887.450 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▆█▁▁▂▂▂▁▂▂▂▁                                                
  █████████████▆▆▆▅▅▄▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▁▂▁▁▁▂ ▃
  2.45 ms         Histogram: frequency by time        6.44 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

My median run time is 2.4ms. You will get full credit if your median run time is within 10ms. The points you will get are

In [17]:
clamp(10 / (median(bm_emupdate).time / 1e6) * 10, 0, 10)

10.0

### Memory

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [18]:
clamp(10 - median(bm_emupdate).memory / 100, 0, 10)

10.0

## Q7. Starting point

We use the same least squares estimates as in HW4 as starting point. 

In [19]:
"""
    init_ls!(m::LmmModel)

Initialize parameters of a `LmmModel` object from the least squares estimate. 
`m.β`, `m.L`, and `m.σ²` are overwritten with the least squares estimates.
"""
function init_ls!(m::LmmModel{T}) where T <: AbstractFloat
    p, q = size(m.data[1].X, 2), size(m.data[1].Z, 2)
    # LS estimate for β
    mul!(m.β, m.xtxinv, m.xty)
    # LS etimate for σ2 and Σ
    rss, ntotal = zero(T), 0
    fill!(m.ztz2, 0)
    fill!(m.ztr2, 0)    
    @inbounds for i in eachindex(m.data)
        obs = m.data[i]
        ntotal += length(obs.y)
        # update Xt * res
        BLAS.gemv!('N', T(-1), obs.xtx, m.β, T(1), copy!(obs.xtr, obs.xty))
        # rss of i-th individual
        rss += obs.yty - dot(obs.xty, m.β) - dot(obs.xtr, m.β)
        # update Zi' * res
        BLAS.gemv!('N', T(-1), obs.ztx, m.β, T(1), copy!(obs.ztr, obs.zty))
        # Zi'Zi ⊗ Zi'Zi
        kron_axpy!(obs.ztz, obs.ztz, m.ztz2)
        # Zi'res ⊗ Zi'res
        kron_axpy!(obs.ztr, obs.ztr, m.ztr2)
    end
    m.σ²[1] = rss / ntotal
    # LS estimate for Σ = LLt
    LAPACK.potrf!('U', m.ztz2)
    BLAS.trsv!('U', 'T', 'N', m.ztz2, m.ztr2)
    BLAS.trsv!('U', 'N', 'N', m.ztz2, m.ztr2)
    copyto!(m.Σ, m.ztr2)
    copy!(m.L, m.Σ)
    LAPACK.potrf!('L', m.L)
    for j in 2:q, i in 1:j-1
        m.L[i, j] = 0
    end
    m
end

"""
    kron_axpy!(A, X, Y)

Overwrite `Y` with `A ⊗ X + Y`. Same as `Y += kron(A, X)` but
more memory efficient.
"""
function kron_axpy!(
        A::AbstractVecOrMat{T},
        X::AbstractVecOrMat{T},
        Y::AbstractVecOrMat{T}
        ) where T <: Real
    m, n = size(A, 1), size(A, 2)
    p, q = size(X, 1), size(X, 2)
    @assert size(Y, 1) == m * p
    @assert size(Y, 2) == n * q
    @inbounds for j in 1:n
        coffset = (j - 1) * q
        for i in 1:m
            a = A[i, j]
            roffset = (i - 1) * p            
            for l in 1:q
                r = roffset + 1
                c = coffset + l
                for k in 1:p                
                    Y[r, c] += a * X[k, l]
                    r += 1
                end
            end
        end
    end
    Y
end

kron_axpy!

In [20]:
init_ls!(lmm)
@show lmm.β
@show lmm.Σ
@show lmm.L
@show lmm.σ²

lmm.β = [0.18207934611476334, 6.500480700993723, -3.4979107842091604, 1.001113296229796, 5.0002519857919285]
lmm.Σ = [1.979430283668505 0.0725846100391669 0.05717147035274013; 0.07258461003916691 1.2840385734767714 -0.0770794276897856; 0.05717147035274013 -0.0770794276897856 0.9509885905046902]
lmm.L = [1.4069222734993234 0.0 0.0; 0.05159105901325531 1.1319792118703693 0.0; 0.04063584138912108 -0.06994463586493141 0.9718256360134829]
lmm.σ² = [5.709004733413663]


1-element Vector{Float64}:
 5.709004733413663

## Q8. Estimation by EM

We write a function `fit!` that implements the EM algorithm for estimating LMM.

In [21]:
"""
    fit!(m::LmmModel)

Fit an `LmmModel` object by MLE using a EM algorithm. Start point 
should be provided in `m.β`, `m.σ²`, `m.L`.
"""
function fit!(
        m       :: LmmModel;
        maxiter :: Integer       = 10_000,
        ftolrel :: AbstractFloat = 1e-12,
        prtfreq :: Integer       = 0
    )
    obj = update_em!(m, true)
    for iter in 0:maxiter
        obj_old = obj
        # EM update
        obj = update_em!(m, false)
        # print obj
        prtfreq > 0 && rem(iter, prtfreq) == 0 && println("iter=$iter, obj=$obj")
        # check monotonicity
        obj < obj_old && (@warn "monotoniciy violated")
        # check convergence criterion
        (obj - obj_old) < ftolrel * (abs(obj_old) + 1) && break
        # warning about non-convergence
        iter == maxiter && (@warn "maximum iterations reached")
    end
    m
end


fit!

## Q9. (20 pts) Test drive

Now we can run our EM algorithm to compute the MLE.

In [22]:
# initialize from least squares
init_ls!(lmm)

@time fit!(lmm, prtfreq = 1);

println("objective value at solution: ", update_em!(lmm)); println()
println("solution values:")
@show lmm.β
@show lmm.σ²
@show lmm.L * transpose(lmm.L)

iter=0, obj=-2.840068882178376e6
iter=1, obj=-2.8400587868036693e6
iter=2, obj=-2.8400587867735964e6
iter=3, obj=-2.8400587867735922e6
  0.091723 seconds (155.67 k allocations: 8.286 MiB, 58.67% compilation time)
objective value at solution: -2.8400587867735922e6

solution values:
lmm.β = [0.18207952873862698, 6.500383547102911, -3.4998642822669384, 0.9997119324206182, 4.999229484613993]
lmm.σ² = [1.4987345519495396]
lmm.L * transpose(lmm.L) = [1.9836319664060416 0.06575197883724945 0.055288404098108375; 0.06575197883724945 1.2814409874856523 -0.09059253368723441; 0.055288404098108375 -0.09059253368723441 0.9434430221723883]


3×3 Matrix{Float64}:
 1.98363     0.065752    0.0552884
 0.065752    1.28144    -0.0905925
 0.0552884  -0.0905925   0.943443

### Correctness

You get 10 points if the following code does not throw `AssertError`.

In [23]:
# objective at solution should be close enough to the optimal
@assert update_em!(lmm) > -2.840059e6

### Efficiency

My median run time 12ms. You get 10 points if your median run time is within 1s.

In [24]:
bm_em = @benchmark fit!($lmm) setup = (init_ls!(lmm))

BenchmarkTools.Trial: 371 samples with 1 evaluation.
 Range (min … max):  11.235 ms … 17.120 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.044 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.104 ms ±  1.263 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▃██▃▁▂    ▂ ▁▁    ▁   █        ▂   ▅      ▂  ▁              
  ███████▅▄▇▆█▄██▇█▇▅█▅▆▆██▇▄▄███▆██▆▆█▄▇▆▇█▇█▆▄█▄▅▄▄▄▄▄▄▃▄▁▃ ▄
  11.2 ms         Histogram: frequency by time        15.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [25]:
# this is the points you get
clamp(1 / (median(bm_em).time / 1e9) * 10, 0, 10)

10.0

## Q10. (10 pts) EM vs Newton type algorithms

Contrast EM algorithm to the Newton type algorithms (gradient free, gradient based, using Hessian) in HW5, in terms of the stability, convergence rate (how fast the algorithm is converging),  final objective value, total run time, derivation, and implementation efforts. 

### Answer

The EM algorithm appears to be much more efficient, with a single run running around 12 ms vs 160 ms with the gradient-based NLopt solver (LD_MMA) in the best case, and in the worst case running 6.77 s with the Ipopt solver. Owing to the greater orders of magnitude in speed and managing to meet its convergence criteria in 4 iterations, it also manages to converge more quickly. It may perhaps be worth noting that the log-likelihood of the EM-solution is about $5 \times 10^{-5}$ units greater than the results from the MixedModels.jl package, which took 1.23 s to run. (While comparably small in terms of relative order of magnitude to the log-likelihood quantities being discussed, Dr. Zhou has formerly claimed that a difference of about $10^{-4}$ was considerable, but in the context of the accuracy of a previous algorithm.) Derivation for both algorithms took some work, however, I have had considerably more practice with the Newton-type algorithms from research than with the EM-algorithm. Adjusting for effort, however, it is possible that the EM-algorithms took somewhat less effort, although at this point, I was already better used to the matrix calculus demanded in Hw5 which I had not yet understood.